In [1]:
import sys, glob, os
from commons import convert_to_json, triq_optimization, qiskit_optimization, \
    calibration_type_enum, qiskit_compilation_enum, normalize_counts, calculate_success_rate_tvd, \
    convert_dict_binary_to_int, convert_dict_int_to_binary, sum_last_n_digits_dict
import wrappers.qiskit_wrapper as qiskit_wrapper
from wrappers.qiskit_wrapper import QiskitCircuit
import pandas as pd
import mthree
import mapomatic as mm
import mthree

from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime.options import SamplerOptions, EstimatorOptions, DynamicalDecouplingOptions, TwirlingOptions

from qiskit_aer import AerSimulator, QasmSimulator, Aer
from qiskit.qasm2 import dumps
import matplotlib.pyplot as plt
import numpy as np

import mitiq
from mitiq import zne, benchmarks

from qEmQUIP import QEM, conf

CB_color_cycle = ['#1F77B4', '#AEC7E8', '#FF7F0E', '#FFBB78', '#2CA02C', '#98DF8A', '#D62728', '#FF9896',
                  '#9467BD', '#C5B0D5', '#8C564B', '#C49C94', '#E377C2', '#F7B6D2', '#7F7F7F', '#C7C7C7',
                  '#BCBD22', '#DBDB8D', '#17BECF', '#9EDAE5'
                  ]

markers = ['o', 'v', '^', 's', '+', '*', 'x', 'd', '<', '>', 'p']
linestyles = ['-', '--', '-.', ':', '-', '--', '-.', ':']
token = "476ea8c61cc54f36e4a21d70a8442f94203c9d87096eaad0886a3e8154d8c2e79bcad6f927c6050a76335dd68d783f478c1b828504748a4377b441c335c831aa"

# select compilation techniques
compilations = ["qiskit_3", "triq_avg_sabre", "triq_lcd_sabre"]

In [2]:
q = QEM(runs=conf.runs, user_id=conf.user_id, token=token, skip_db=True)

Connecting to quantum service...
Saving IBM Account...
Retrieving the real backend information...
Time for setup the services: 22.801740534996497 seconds


In [3]:
qasm_files = q.get_qasm_files_from_path("./circuits/polar_sim/n4/")
print(qasm_files)

q = QEM(runs=conf.runs, user_id=conf.user_id, token=token, skip_db=False)


['./circuits/polar_sim/n4/polar_all_meas_n4.qasm', './circuits/polar_sim/n4/polar_all_meas_n4_x.qasm']
Connecting to quantum service...
Saving IBM Account...
Retrieving the real backend information...
Time for setup the services: 7.067022876988631 seconds


In [4]:
q.update_hardware_configs()

shots = 5000
q.set_backend(program_type="sampler", shots=shots)
qiskit_wrapper.generate_new_props(q.backend, "avg")
qiskit_wrapper.generate_new_props(q.backend, "mix")
qiskit_wrapper.generate_new_props(q.backend, "recent_15_adjust")

Time for update hardware configs: 165.10653252599877 seconds
Time for setup the backends: 0.00010798100265674293 seconds


In [5]:
q = QEM(runs=conf.runs, user_id=conf.user_id, token=token)

qasm_files = q.get_qasm_files_from_path("./circuits/polar_sim/n3")
noise_levels = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
shots = 10000

q.set_backend(program_type="sampler", shots=shots)
q.run_simulator("sampler", qasm_files, compilations, noise_levels, shots, send_to_db=True)

Connecting to quantum service...
Saving IBM Account...
Retrieving the real backend information...
Time for setup the services: 10.813138753990643 seconds
Time for setup the backends: 9.843699808698148e-05 seconds
Start running the simulator...
Time for running the init header: 0.0003690669982461259 seconds
qiskit_3 0.1
Time for setup the backends: 0.00025155500043183565 seconds
qiskit_3 0.2
Time for setup the backends: 0.001430303993402049 seconds
qiskit_3 0.4
Time for setup the backends: 0.0019706290040630847 seconds
qiskit_3 0.6
Time for setup the backends: 0.0022958939953241497 seconds
qiskit_3 0.8
Time for setup the backends: 0.0021174730063648894 seconds
qiskit_3 1.0
Time for setup the backends: 0.002438932002405636 seconds
triq_avg_sabre 0.1
Time for setup the backends: 0.0025962469953810796 seconds
triq_avg_sabre 0.2
Time for setup the backends: 0.0024059879942797124 seconds
triq_avg_sabre 0.4
Time for setup the backends: 0.0024536519922548905 seconds
triq_avg_sabre 0.6
Time for

In [7]:
q = QEM(runs=conf.runs, user_id=conf.user_id, token=token, skip_db=False)
qasm_files = q.get_qasm_files_from_path("./circuits/polar_sim/n4")
noise_levels = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
shots = 10
q.set_backend(program_type="sampler", shots=shots)

Connecting to quantum service...
Saving IBM Account...
Retrieving the real backend information...
Time for setup the services: 8.452421510010026 seconds
Time for setup the backends: 8.680600149091333e-05 seconds


In [8]:
q.run_simulator("sampler", qasm_files, compilations, noise_levels, shots, send_to_db=True)

Start running the simulator...
Time for running the init header: 0.0003460200096014887 seconds
qiskit_3 0.1
Time for setup the backends: 8.589199569541961e-05 seconds
qiskit_3 0.2
Time for setup the backends: 0.002494005995686166 seconds
qiskit_3 0.4
Time for setup the backends: 0.001877678994787857 seconds
qiskit_3 0.6
Time for setup the backends: 0.002583528999821283 seconds
qiskit_3 0.8
Time for setup the backends: 0.0024434749939246103 seconds
qiskit_3 1.0
Time for setup the backends: 0.0025789889914449304 seconds
triq_avg_sabre 0.1
Time for setup the backends: 0.0026748210075311363 seconds
triq_avg_sabre 0.2
Time for setup the backends: 0.0025411770038772374 seconds
triq_avg_sabre 0.4
Time for setup the backends: 0.00260886900650803 seconds
triq_avg_sabre 0.6
Time for setup the backends: 0.0031963749934220687 seconds
triq_avg_sabre 0.8
Time for setup the backends: 0.0024699359928490594 seconds
triq_avg_sabre 1.0
Time for setup the backends: 0.0028142640076112 seconds
triq_lcd_sabr

In [ ]:
q.get_qiskit_result()

Pending jobs:  2
Checking results for:  simulator with header id : 210
Preparing the noisy simulator qiskit_3 0.1
